In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split 
import seaborn as sns
from matplotlib import pyplot as plt 

In [2]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard
from datetime import datetime
from packaging import version

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."
import tensorboard
tensorboard.__version__

TensorFlow version:  2.6.0


'2.6.0'

In [3]:
rooms = {
    "Brayden's Room": 0,
    "Bedroom": 1,
    "Bathroom 1": 2,
    "Kitchen": 3,
    "Dining Room": 4,
    "Living Room": 5,
    "Extra Room": 6
}

rooms_labels = dict()
for key, value in rooms.items():
    rooms_labels[value] = key

## Inport and clean data

In [4]:
ble = pd.read_csv("data/ble_data-3.csv", delimiter=",", quoting = 3)

In [5]:
ble = ble.fillna(-200)

replacements = dict()
for column in ble.columns:
    replacements[column] = column.replace(':', '-')
    
    if column != "room":
        ble[column] = pd.to_numeric(ble[column])


ble.rename(columns=replacements, inplace=True)
ble_shuffle =  ble.sample(frac=1).reset_index(drop=True)

# Normalize and drop unused columns
drop = []
for column in ble_shuffle.columns:
    if column == "room":
        continue
    if ble_shuffle[column].max() == ble_shuffle[column].min():
        drop.append(column)

ble_cleaned = ble_shuffle.drop(columns=drop)

ble_normalized = ble_cleaned.copy()

# apply normalization techniques
for column in ble_normalized.columns:
    if column == "room":
        ble_normalized[column] = ble_normalized[column]
    else:
        ble_normalized[column] = (ble_normalized[column] - ble_normalized[column].min()) / (ble_normalized[column].max() - ble_normalized[column].min())    



In [6]:
train_df, test_df = train_test_split(ble_normalized, test_size=0.1)
train_df, val_df = train_test_split(train_df, test_size=0.2)

print(len(train_df), 'train examples')
print(len(val_df), 'validation examples')
print(len(test_df), 'test examples')

13687 train examples
3422 validation examples
1901 test examples


## Prepare Model

In [7]:
# Create an empty list
feature_cols = []

# Numeric Columns
for header in ble_normalized.columns:
    if header == "room":
        continue
    
    feature_cols.append(tf.feature_column.numeric_column(header))

print("Feature columns: ", feature_cols, "\n")

feature_layer = tf.keras.layers.DenseFeatures(feature_cols, name='Features')

Feature columns:  [NumericColumn(key='50-de-06-6c-21-08', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='e4-f8-9d-6c-92-2c', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='4d-74-04-a1-d0-66', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='f5-3a-4f-36-13-5f', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='28-11-a5-d6-8b-07', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='77-e2-15-65-85-90', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='d2-f5-db-e4-dc-65', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='12-4e-8f-cf-e8-de', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='57-31-a2-ac-c3-16', shape=(1,), default_value=None, dtype=tf.float32, normalizer_f

In [8]:
# Hyperparameters
epochs = 100

# Label 
label_name = "room"
shuffle = True

#---Create a sequential model---#
model = tf.keras.Sequential([
    feature_layer,
    # tf.keras.layers.Dense(256 ,activation='relu', name='hidden_layer'),
    tf.keras.layers.Dense(60 ,activation='relu', name='hidden_layer2'),
    tf.keras.layers.Dense(20 ,activation='relu', name='hidden_layer3'),
    tf.keras.layers.Dense(len(rooms), name='output')

])


model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


#---Train the Model---#
# Keras TensorBoard callback.
logdir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)


# Split the datasets into features and label.
train_lbl = np.array([rooms[x] for x in train_df[label_name]])
train_df_ = train_df.drop(columns=[label_name])
train_ft = {name:np.array(value) for name, value in train_df_.items()}

val_lbl =  np.array([rooms[x] for x in val_df[label_name]])
val_df_ = val_df.drop(columns=[label_name])
val_ft = {name:np.array(value) for name, value in val_df_.items()}

model.fit(train_ft, train_lbl, epochs=epochs
          ,validation_data=(val_ft, val_lbl)
         )


2021-11-20 12:06:10.008674: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-20 12:06:10.025708: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-20 12:06:10.025725: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-11-20 12:06:10.025853: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-11-20 12:06:10.191623: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
408/428 [===========================>..] - ETA: 0s - loss: 1.0751 - accuracy: 0.6133WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'50-de-06-6c-21-08': <tf.Tensor 'ExpandDims_21:0' shape=(None, 1) dtype=float32>, 'e4-f8-9d-6c-92-2c': <tf.Tensor 'ExpandDims_60:0' shape=(None, 1) dtype=float32>, '4d-74-04-a1-d0-66': <tf.Tensor 'ExpandDims_19:0' shape=(None, 1) dtype=float32>, 'f5-3a-4f-36-13-5f': <tf.Tensor 'ExpandDims_67:0' shape=(None, 1) dtype=float32>, '28-11-a5-d6-8b-07': <tf.Tensor 'ExpandDims_8:0' shape=(None, 1) dtype=float32>, '77-e2-15-65-85-90': <tf.Tensor 'ExpandDims_41:0' shape=(None, 1) dtype=float32>, 'd2-f5-db-e4-dc-65': <tf.Tensor 'ExpandDims_53:0' shape=(None, 1) dtype=float32>, '12-4e-8f-cf-e8-de': <tf.Tensor 'ExpandDims_5:0' shape=(None, 1) dtype=float32>, '57-31-a2-ac

In [9]:
test_lbl = np.array([rooms[x] for x in test_df[label_name]])
test_df_ = test_df.drop(columns=[label_name])
test_ft = {key:np.array(value) for key, value in test_df_.items()}

model.evaluate(test_ft, test_lbl)

60/60 [==============================] - 0s 2ms/step - loss: 0.2511 - accuracy: 0.8927


[0.25112712383270264, 0.8926880359649658]

In [16]:
# Convert to softmax as probability distrbution 
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

In [17]:
def predict(data):
    converted = convert(data)
    predicted = probability_model.predict(converted)
    arg_max = np.argmax(predicted)
    room = rooms_labels[arg_max]
    prob = predicted[0][arg_max]
    return room, prob

def convert(data):
    new_data = dict()
    for column in train_df_.columns:
        val = np.array([data.get(column, -200)])
        # Normalize
        new_data[column] = (val - ble_cleaned[column].min()) / (ble_cleaned[column].max() - ble_cleaned[column].min())  
    return new_data

In [ ]:
import socket
import json
import traceback

def main():
   
    host = '10.0.0.132' #Server ip
    port = 4004

    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    s.bind((host, port))

    print("Server Started")
    try:
        while True:
            data, addr = s.recvfrom(10240)
            try:
                data_loaded = json.loads(data) #data loaded
            except json.decoder.JSONDecodeError:
                print("Json error with {}".format(data))
            label, prob = predict(data_loaded)
            predicted_data = {"room": label, "prob": str(prob)}
            message = json.dumps(predicted_data) #data serialized
            s.sendto(message.encode(), addr)
    except KeyboardInterrupt as ki:
        pass
    except Exception as e:
        traceback.print_exc()
    finally:
        print("Ending")
        s.close()

main()

Server Started
